In [4]:
import pandas as pd
data = pd.read_json('ft_data_3.json')
data

,prompt,content,label,token
0,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-10-10 09:00:00+08:00\n时间：2018-10-09 17...,极度负面,2291
1,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-10-24 09:00:00+08:00\n时间：2018-10-09 17...,极度负面,3099
2,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-12-07 09:00:00+08:00\n时间：2018-12-06 18...,负面,4949
3,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-12-24 09:00:00+08:00\n时间：2018-12-06 18...,负面,6829
4,请根据以下新闻文本，预测三一重工股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2018-12-24 09:00:00+08:00\n时间：2018-12-06 18...,负面,8136
...,...,...,...,...
5082,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-29 19...,正面,4210
5083,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-30 18...,正面,6582
5084,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-30 18...,正面,7217
5085,请根据以下新闻文本，预测中国建筑股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正...,现在是2023-03-31 09:00:00+08:00\n时间：2023-03-30 18...,正面,7687


In [5]:
data['label'].value_counts()

label
中性      1031
极度负面    1027
极度正面    1018
负面      1013
正面       998
Name: count, dtype: int64

In [6]:
import pandas as pd
import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
from modelscope import GenerationConfig
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from tqdm import tqdm
import json
from llmtuner import ChatModel


def predict(model, prompt):
    query = prompt
    messages = [{"role": "user", "content": query}]
    resp = model.chat(messages, system="你需要帮助我判断股票的对数收益率情况，请选出其中的正确答案。")
    return (resp[0].response_text)


def evaluate_accuracy(model, test_data):
    results = []
    correct_predictions = 0
    total_predictions = 0
    with tqdm(total=test_data.shape[0]) as pbar:
        for _, item in test_data.iterrows():
            text_input = f'''{item['prompt']}\n{item['content']}'''
            predicted_response = predict(model, text_input)

            expected_response = item['expected']
            results.append({
                "prompt": item['prompt'],
                "content": item['content'],
                "expected": expected_response,
                "predicted": predicted_response
            })
            total_predictions += 1
            if expected_response in predicted_response:
                correct_predictions += 1

            accuracy = correct_predictions / total_predictions
            pbar.set_description(f"Accuracy: {accuracy:.2f}")
            pbar.update()

    with open('test_dataset/evaluation_results_qwen.json', 'w') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

    return accuracy


max_samples = 100
test_data = pd.read_json('test_dataset/results.json')
test_data = test_data.sample(frac=1).reset_index(drop=True)
test_data = test_data[:max_samples]
model_path = "../exp_model/v2"
# model_path = "/home/zhangmin/.cache/modelscope/hub/qwen/Qwen-7B-Chat"
model_path = "/home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat"


model = ChatModel(dict(
    model_name_or_path=model_path,
    template="qwen",
    temperature=0.5,
    cutoff_len=8192,
    infer_backend="vllm",
    # top_k=10,
    adapter_name_or_path='../train/saves/v2_14b',
))

accuracy = evaluate_accuracy(model, test_data)
print("Accuracy:", accuracy)

[INFO|configuration_utils.py:724] 2024-05-05 00:40:35,975 >> loading configuration file /home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat/config.json
[INFO|configuration_utils.py:724] 2024-05-05 00:40:35,978 >> loading configuration file /home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat/config.json
[INFO|configuration_utils.py:789] 2024-05-05 00:40:35,979 >> Model config QWenConfig {
  "_name_or_path": "/home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat",
  "architectures": [
    "QWenLMHeadModel"
  ],
  "attn_dropout_prob": 0.1,
  "auto_map": {
    "AutoConfig": "configuration_qwen.QWenConfig",
    "AutoModelForCausalLM": "modeling_qwen.QWenLMHeadModel"
  },
  "bf16": true,
  "emb_dropout_prob": 0.1,
  "fp16": false,
  "fp32": false,
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27392,
  "kv_channels": 128,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 16384,
  "model_t

05/05/2024 00:40:36 - INFO - llmtuner.data.template - Add eos token: <|im_end|>
05/05/2024 00:40:36 - INFO - llmtuner.data.template - Add pad token: <|im_end|>


[INFO|configuration_utils.py:724] 2024-05-05 00:40:36,613 >> loading configuration file /home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat/config.json
[INFO|configuration_utils.py:724] 2024-05-05 00:40:36,615 >> loading configuration file /home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat/config.json
[INFO|configuration_utils.py:789] 2024-05-05 00:40:36,616 >> Model config QWenConfig {
  "_name_or_path": "/home/zhangmin/.cache/modelscope/hub/TongyiFinance/Tongyi-Finance-14B-Chat",
  "architectures": [
    "QWenLMHeadModel"
  ],
  "attn_dropout_prob": 0.1,
  "auto_map": {
    "AutoConfig": "configuration_qwen.QWenConfig",
    "AutoModelForCausalLM": "modeling_qwen.QWenLMHeadModel"
  },
  "bf16": true,
  "emb_dropout_prob": 0.1,
  "fp16": false,
  "fp32": false,
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27392,
  "kv_channels": 128,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 16384,
  "model_t

KeyboardInterrupt: 